In [1]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# GeoTiff Images
import rasterio
from osgeo import gdal

# Visualisation
from matplotlib import pyplot as plt
import matplotlib.image as img
from matplotlib.pyplot import figure
from PIL import Image
%matplotlib inline

# Model Building
import ultralytics
from ultralytics import YOLO
import labelme2yolo

# Others
import os
import shutil
import zipfile
import csv
from tqdm import tqdm

In [2]:
# !labelme2yolo --json_dir ./training_data_8/

In [3]:
# def copy_files(source_dir, dest_dir):
#     for filename in tqdm(os.listdir(source_dir)):
#         if os.path.isfile(os.path.join(source_dir, filename)):
#             source_path = os.path.join(source_dir, filename)
#             destination_path = os.path.join(dest_dir, filename)
#             shutil.copy2(source_path, destination_path)

In [4]:
#copy images
# source_train = os.getcwd() + '/training_data_8/YOLODataset/images/train'
# source_val = os.getcwd() + '/training_data_8/YOLODataset/images/val'

# train_dir = os.getcwd() + '/training_data_8/YOLODataset/datasets/train'
# val_dir = os.getcwd() + '/training_data_8/YOLODataset/datasets/val'
# os.makedirs(train_dir, exist_ok=True)
# os.makedirs(val_dir, exist_ok=True)

# copy_files(source_train, train_dir)
# copy_files(source_val, val_dir)

In [5]:
#copy labels
# source_train = os.getcwd() + '/training_data_8/YOLODataset/labels/train'
# source_val = os.getcwd() + '/training_data_8/YOLODataset/labels/val'
# copy_files(source_train, train_dir)
# copy_files(source_val, val_dir)

In [6]:
#Loading the model
# model = YOLO('yolov8n.pt')

In [7]:
# Display model information (optional)
# model.info()

In [8]:
#changing yaml directory links
# with open(os.getcwd()+'/training_data_8/YOLODataset/dataset.yaml', 'r') as file:
#     lines = file.readlines()
# lines[0] = 'train: ./datasets/train' + '\n'
# lines[1] = 'val: ./datasets/val' + '\n'
# with open(os.getcwd()+'/training_data_8/YOLODataset/dataset.yaml', 'w') as file:
#     file.writelines(lines)

In [9]:
# Train the model on the dataset for recommended initial 300 epochs
# all_data_yaml = os.getcwd()+'/training_data_8/YOLODataset/dataset.yaml'
# results = model.train(data=all_data_yaml, epochs=300, imgsz=512)

In [10]:
# restart the model

# Load the last model
model = YOLO('./runs/detect/train/weights/last.pt')  # load a partially trained model

In [ ]:
# Resume training
results = model.train(resume=True)

Ultralytics YOLOv8.1.24 🚀 Python-3.11.4 torch-2.2.1+cu121 CPU (Intel Xeon E5-2673 v4 2.30GHz)
engine/trainer: task=detect, mode=train, model=runs/detect/train/weights/last.pt, data=/home/jovyan/PlanetaryComputerExamples/EYOpenScience/model_development_8/training_data_8/YOLODataset/dataset.yaml, epochs=300, time=None, patience=100, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=0, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, sh

train: Scanning /home/jovyan/PlanetaryComputerExamples/EYOpenScience/model_development_8/training_data_8/YOLODataset/datasets/train.cache... 1480 images, 33 backgrounds, 0 corrupt: 100%|██████████| 1480/1480 [00:00<?, ?it/s]
val: Scanning /home/jovyan/PlanetaryComputerExamples/EYOpenScience/model_development_8/training_data_8/YOLODataset/datasets/val.cache... 370 images, 2 backgrounds, 0 corrupt: 100%|██████████| 370/370 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Resuming training from runs/detect/train/weights/last.pt from epoch 51 to 300 total epochs
Image sizes 512 train, 512 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     51/300         0G      1.099     0.8183      1.044        337        512:  15%|█▌        | 14/93 [08:19<1:00:11, 45.71s/it]

In [2]:
# Load the Model
model = YOLO('./runs/detect/train/weights/best.pt')

In [5]:
# Decoding according to the .yaml file class names order
decoding_of_predictions ={0: 'undamagedresidentialbuilding', 1: 'damagedresidentialbuilding', 2: 'undamagedcommercialbuilding', 3: 'damagedcommercialbuilding'}

directory = 'challenge_1_submission_images/'
# Directory to store outputs
results_directory = './Validation_Data_Results'

# Create submission directory if it doesn't exist
if not os.path.exists(results_directory):
    os.makedirs(results_directory)

# Loop through each file in the directory
for filename in os.listdir(directory):
    # Check if the current object is a file and ends with .jpeg
    if os.path.isfile(os.path.join(directory, filename)) and filename.lower().endswith('.jpg'):
        # Perform operations on the file
        file_path = os.path.join(directory, filename)
        print(file_path)
        print("Making a prediction on ", filename)
        results = model.predict(file_path, save=True, iou=0.5, save_txt=True, conf=0.25)
        
        for r in results:
            conf_list = r.boxes.conf.numpy().tolist()
            clss_list = r.boxes.cls.numpy().tolist()
            original_list = clss_list
            updated_list = []
            for element in original_list:
                 updated_list.append(decoding_of_predictions[int(element)])

        bounding_boxes = r.boxes.xyxy.numpy()
        confidences = conf_list
        class_names = updated_list

        # Check if bounding boxes, confidences and class names match
        if len(bounding_boxes) != len(confidences) or len(bounding_boxes) != len(class_names):
            print("Error: Number of bounding boxes, confidences, and class names should be the same.")
            continue
        text_file_name = os.path.splitext(filename)[0]
        # Creating a new .txt file for each image in the submission_directory
        with open(os.path.join(results_directory, f"{text_file_name}.txt"), "w") as file:
            for i in range(len(bounding_boxes)):
                # Get coordinates of each bounding box
                left, top, right, bottom = bounding_boxes[i]
                # Write content to file in desired format
                file.write(f"{class_names[i]} {confidences[i]} {left} {top} {right} {bottom}\n")
        print("Output files generated successfully.")

challenge_1_submission_images/Validation_Post_Event_008.jpg
Making a prediction on  Validation_Post_Event_008.jpg

image 1/1 /home/jovyan/PlanetaryComputerExamples/EYOpenScience/model_development_8/challenge_1_submission_images/Validation_Post_Event_008.jpg: 512x512 13 undamagedresidentialbuildings, 2 damagedresidentialbuildings, 4 undamagedcommercialbuildings, 2 damagedcommercialbuildings, 2568.5ms
Speed: 20.0ms preprocess, 2568.5ms inference, 2.3ms postprocess per image at shape (1, 3, 512, 512)
Results saved to runs/detect/predict
1 label saved to runs/detect/predict/labels
Output files generated successfully.
challenge_1_submission_images/Validation_Post_Event_002.jpg
Making a prediction on  Validation_Post_Event_002.jpg

image 1/1 /home/jovyan/PlanetaryComputerExamples/EYOpenScience/model_development_8/challenge_1_submission_images/Validation_Post_Event_002.jpg: 512x512 9 undamagedresidentialbuildings, 6 damagedresidentialbuildings, 1 undamagedcommercialbuilding, 2 damagedcommerci

In [6]:
# Define your source directory and the destination where the zip file will be created
source_dir = results_directory
destination_zip = 'submission'

# Create a zip file from the directory
shutil.make_archive(destination_zip, 'zip', source_dir)

print(f"Directory {source_dir} has been successfully zipped into {destination_zip}.")

Directory ./Validation_Data_Results has been successfully zipped into submission.
